In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
df_movies = pd.read_pickle("../../data/ml-1m-after_eda/movies.pickle")
df_users = pd.read_pickle("../../data/ml-1m-after_eda/users.pickle")
df_ratings = pd.read_pickle("../../data/ml-1m-after_eda/ratings.pickle")

In [11]:
df_users.head()

,UserID,Gender,Age,Occupation,Zip-Code,State,Latitude,Longitude
0,1,F,1,10,48067,MI,42.488735,-83.13752
1,2,M,56,16,70072,LA,29.869283,-90.10933
2,3,M,25,15,55117,MN,44.989065,-93.10666
3,4,M,45,7,02460,MA,42.352996,-71.20907
4,5,M,25,20,55455,MN,44.971965,-93.23588


In [12]:
df_movies.head()

,MovieID,Title,Genres,Year,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,"[Animation, Children's, Comedy]",1995,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,"[Comedy, Drama]",1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,[Comedy],1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp,Datetime,Date
0,1,1193,5,978300760,2000-12-31 22:12:40,2000-12-31
1,1,661,3,978302109,2000-12-31 22:35:09,2000-12-31
2,1,914,3,978301968,2000-12-31 22:32:48,2000-12-31
3,1,3408,4,978300275,2000-12-31 22:04:35,2000-12-31
4,1,2355,5,978824291,2001-01-06 23:38:11,2001-01-06


In [119]:
def item_item_filtering(user_id):
    user_movie_matrix = df_ratings.pivot_table(index='UserID', columns='MovieID', values='Rating',fill_value=0)

    sim_matrix = cosine_similarity(user_movie_matrix)
    current_user = sim_matrix[user_id]
    index_of_similarties = current_user.argsort()[::-1][:10]

    films_to_recommend = set()
    for similar_user_index in index_of_similarties:
        film_rates = user_movie_matrix.iloc[similar_user_index]
        for film_index, film_rate in film_rates.items():
            if user_movie_matrix.iloc[user_id][film_index] == 0 and film_rate != 0:
                films_to_recommend.add(film_index)
    
    # Filter out only good films
    filtered_recommendations = set()

    for film_index in films_to_recommend:
        for similar_user_index in index_of_similarties:
            film = df_ratings[df_ratings['MovieID'] == film_index]
            all_film_rates = []
            if similar_user_index < len(film['Rating']):
                rating = film['Rating'].iloc[similar_user_index]
                all_film_rates.append(rating)

        average_rating = np.mean(all_film_rates)
        if average_rating >= 4:
            filtered_recommendations.add(film_index)

    print(filtered_recommendations)

            
    

In [120]:
item_item_filtering(20)

/Users/yuriivoievidka/MLProjects/IT-jim/Test Task/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/yuriivoievidka/MLProjects/IT-jim/Test Task/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{260, 2571, 780, 2716, 541, 1193, 2858, 1196, 1197, 1198, 1580, 1200, 50, 318, 1214, 1097, 2762, 589, 1617, 1240, 858, 2396, 480, 356, 2916, 2791, 1127, 2028, 110, 1265, 1270}


In [122]:
df_movies[df_movies['MovieID'] == 648]

,MovieID,Title,Genres,Year,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
642,648,Mission: Impossible,"[Action, Adventure, Mystery]",1996,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [123]:
df_movies[df_movies['MovieID'] == 260]

,MovieID,Title,Genres,Year,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
257,260,Star Wars: Episode IV - A New Hope,"[Action, Adventure, Fantasy, Sci-Fi]",1977,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [121]:
df_ratings[df_ratings['UserID'] == 20]

,UserID,MovieID,Rating,Timestamp,Datetime,Date
2500,20,648,4,978143369,2000-12-30 02:29:29,2000-12-30
2501,20,3863,3,978143355,2000-12-30 02:29:15,2000-12-30
2502,20,589,4,978143508,2000-12-30 02:31:48,2000-12-30
2503,20,1694,3,1009669071,2001-12-29 23:37:51,2001-12-29
2504,20,2641,4,1009669115,2001-12-29 23:38:35,2001-12-29
2505,20,1912,5,978143508,2000-12-30 02:31:48,2000-12-30
2506,20,2571,5,978143508,2000-12-30 02:31:48,2000-12-30
2507,20,3527,4,1009669181,2001-12-29 23:39:41,2001-12-29
2508,20,1923,4,978143576,2000-12-30 02:32:56,2000-12-30
2509,20,1371,4,1009669227,2001-12-29 23:40:27,2001-12-29


## Finding Similarity